In [ ]:
# importing modules


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision.utils import make_grid, save_image

In [ ]:
# setting hyperparameters


img_size = 64
batch_size = 100
num_threads = 8
lat_dim = 10

beta = 4

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
# loading dataset that is downloaded from the public


dataset = np.load('data/dsprites/dsprites.npz', allow_pickle=True, encoding='latin1')
imgs = dataset['imgs']
classes = dataset['latents_classes']
latent_sizes = dataset['metadata'][()]['latents_sizes']

In [ ]:
# generating samples where instances within a batch share the same factor of variation.


base = torch.zeros(size=(1, img_size * img_size + 1))
for j in range(1500):
    factor = np.random.choice(np.arange(5), 1)
    num_cand = latent_sizes[factor+1]
    value = np.random.choice(np.arange(num_cand), 1)
    index = classes[:,factor+1]==value
    index = index.reshape(-1)
        
    img = torch.Tensor(imgs[index])
    num = len(img)
    indices = torch.randperm(num)
    img = img[indices][:batch_size].view(-1, img_size * img_size)
    label = torch.Tensor([factor] * batch_size).view(-1, 1)
    samples = torch.cat((img, label), dim=1)
    base = torch.cat((base , samples), dim=0)
    torch.save(base[1:], 'data/dsprites/base.pt')     
print('complete')